In [1]:
import ccxt
import pandas as pd
import time
import sys

In [2]:
from config_account_reader import ConfigAccountReader
config_accout_reader = ConfigAccountReader('config_account.yaml')
exchange_name = config_accout_reader.get_exchange_name()
symbol = config_accout_reader.get_market_symbol()
public_key = config_accout_reader.get_public_key()
secret = config_accout_reader.get_secret_key()

# I need the exchange 

#### https://docs.ccxt.com/en/latest/manual.html#configuring-api-keys

In [3]:
# from variable id
exchange_id = exchange_name
exchange_class = getattr(ccxt, exchange_id)
exchange = exchange_class({
    'apiKey': public_key,
    'secret': secret,
})

In [4]:
ticker = exchange.fetch_ticker(symbol)
print(ticker)

{'symbol': 'XLM/USDT', 'timestamp': 1669558128458, 'datetime': '2022-11-27T14:08:48.458Z', 'high': 0.0911, 'low': 0.0873, 'bid': 0.09, 'bidVolume': 254590.0, 'ask': 0.0901, 'askVolume': 78769.0, 'vwap': 0.08946208, 'open': 0.0895, 'close': 0.0901, 'last': 0.0901, 'previousClose': 0.0896, 'change': 0.0006, 'percentage': 0.67, 'average': 0.0898, 'baseVolume': 55163749.0, 'quoteVolume': 4935063.4861, 'info': {'symbol': 'XLMUSDT', 'priceChange': '0.00060000', 'priceChangePercent': '0.670', 'weightedAvgPrice': '0.08946208', 'prevClosePrice': '0.08960000', 'lastPrice': '0.09010000', 'lastQty': '746.00000000', 'bidPrice': '0.09000000', 'bidQty': '254590.00000000', 'askPrice': '0.09010000', 'askQty': '78769.00000000', 'openPrice': '0.08950000', 'highPrice': '0.09110000', 'lowPrice': '0.08730000', 'volume': '55163749.00000000', 'quoteVolume': '4935063.48610000', 'openTime': '1669471728458', 'closeTime': '1669558128458', 'firstId': '133688269', 'lastId': '133706746', 'count': '18478'}}


In [5]:
from config_strategy_reader import ConfigStrategyReader
config_strategy_reader = ConfigStrategyReader('config_strategy.yaml')

position_size = config_strategy_reader.get_position_size()
check_orders_frequency = config_strategy_reader.get_check_orders_frequency()
closed_order_status = config_strategy_reader.get_closed_order_status()

num_buy_grid_lines = config_strategy_reader.get_num_buy_grid_lines()
num_sell_grid_lines = config_strategy_reader.get_num_sell_grid_lines()
grid_size = config_strategy_reader.get_grid_size()

In [6]:
# initial_buy_order = exchange.create_market_buy_order(symbol, position_size*num_sell_grid_lines)

In [7]:
# initial_buy_order

In [8]:
buy_orders = []

In [9]:
for i in range(num_buy_grid_lines):
    price = float(ticker['bid']) - (grid_size * (i + 1))
    print(f'submitting limit BUY order at {price:.4f}')
    order = exchange.create_limit_buy_order(symbol, position_size, price)
    buy_orders.append(order)

submitting limit BUY order at 0.0892
submitting limit BUY order at 0.0884
submitting limit BUY order at 0.0876
submitting limit BUY order at 0.0868


In [10]:
sell_orders = []

In [11]:
for i in range(num_sell_grid_lines):
    price = float(ticker['bid']) + (grid_size * (i + 1))
    print(f'submitting limit SELL order at {price:.4f}')
    order = exchange.create_limit_sell_order(symbol, position_size, price)
    sell_orders.append(order)

submitting limit SELL order at 0.0908
submitting limit SELL order at 0.0916
submitting limit SELL order at 0.0924
submitting limit SELL order at 0.0932


In [12]:
def try_fetch_order(id_order, symbol):
    while True:
        try:
            order = exchange.fetch_order(id_order, symbol)
            return order
        except Exception as ex:
            print(f'Exception occured {ex}')
            time.sleep(check_orders_frequency)
            continue   

In [13]:
while True:

    closed_order_ids = []
    
    for buy_order in buy_orders:
        
        print(f"Checking for open buy orders {buy_order['id']}")
        
        order = try_fetch_order(buy_order['id'], symbol)
        
        order_info = order['info']
        print(f"status: {order_info['status']}")
        if order_info['status'] == closed_order_status:
            closed_order_ids.append(order['id'])
            print(f'BUY order executed at {order_info["price"]}')
            new_sell_price = float(order_info['price']) + grid_size
            print(f'creating new limit sell order at {new_sell_price:.4f}')
            new_sell_order = exchange.create_limit_sell_order(symbol, position_size, new_sell_price)
            sell_orders.append(new_sell_order)
            
        time.sleep(check_orders_frequency)
    
    for sell_order in sell_orders:
        
        print(f'Checking for open buy orders {sell_order["id"]}')
        
        order = try_fetch_order(sell_order['id'], symbol)
        
        order_info = order['info']
        print(f"status: {order_info['status']}")
        if order_info['status'] == closed_order_status:
            closed_order_ids.append(order['id'])
            print(f'SELL order executed at {order_info["price"]}')
            new_buy_price = float(order_info["price"]) - grid_size
            print(f'creating new limit buy order at {new_buy_price:.4f}')
            new_buy_order = exchange.create_limit_buy_order(symbol, position_size, new_buy_price)
            buy_orders.append(new_buy_order)
        
        time.sleep(check_orders_frequency)
            
    for order_id in closed_order_ids:
        buy_orders = [buy_order for buy_order in buy_orders if buy_order['id'] != order_id]
        sell_orders = [sell_order for sell_order in sell_orders if sell_order['id'] != order_id]
        
    if len(sell_orders)  == 0:
        sys.exit('stopping bot, nothing left to sell')  
    

Checking for open buy orders 1608253127
status: NEW
Checking for open buy orders 1608253128
status: NEW
Checking for open buy orders 1608253129
status: NEW
Checking for open buy orders 1608253130
status: NEW
Checking for open buy orders 1608253131
status: NEW
Checking for open buy orders 1608253133
status: NEW
Checking for open buy orders 1608253135
status: NEW
Checking for open buy orders 1608253136
status: NEW
Checking for open buy orders 1608253127
status: NEW
Checking for open buy orders 1608253128
status: NEW
Checking for open buy orders 1608253129
status: NEW
Checking for open buy orders 1608253130
status: NEW
Checking for open buy orders 1608253131
status: NEW
Checking for open buy orders 1608253133
status: NEW
Checking for open buy orders 1608253135
status: NEW
Checking for open buy orders 1608253136
status: NEW
Checking for open buy orders 1608253127
status: NEW
Checking for open buy orders 1608253128
status: NEW
Checking for open buy orders 1608253129
status: NEW
Checking for

status: NEW
Checking for open buy orders 1608253135
status: NEW
Checking for open buy orders 1608253136
status: NEW
Checking for open buy orders 1608253127
status: NEW
Checking for open buy orders 1608253128
status: NEW
Checking for open buy orders 1608253129
status: NEW
Checking for open buy orders 1608253130
status: NEW
Checking for open buy orders 1608253131
status: NEW
Checking for open buy orders 1608253133
status: NEW
Checking for open buy orders 1608253135
status: NEW
Checking for open buy orders 1608253136
status: NEW
Checking for open buy orders 1608253127
status: NEW
Checking for open buy orders 1608253128
status: NEW
Checking for open buy orders 1608253129
status: NEW
Checking for open buy orders 1608253130
status: NEW
Checking for open buy orders 1608253131
status: NEW
Checking for open buy orders 1608253133
status: NEW
Checking for open buy orders 1608253135
status: NEW
Checking for open buy orders 1608253136
status: NEW
Checking for open buy orders 1608253127
status: NEW


status: NEW
Checking for open buy orders 1608253131
status: NEW
Checking for open buy orders 1608253133
status: NEW
Checking for open buy orders 1608253135
status: NEW
Checking for open buy orders 1608253136
status: NEW
Checking for open buy orders 1608253127
status: NEW
Checking for open buy orders 1608253128
status: NEW
Checking for open buy orders 1608253129
status: NEW
Checking for open buy orders 1608253130
status: NEW
Checking for open buy orders 1608253131
status: NEW
Checking for open buy orders 1608253133
status: NEW
Checking for open buy orders 1608253135
status: NEW
Checking for open buy orders 1608253136
status: NEW
Checking for open buy orders 1608253127
status: NEW
Checking for open buy orders 1608253128
status: NEW
Checking for open buy orders 1608253129
status: NEW
Checking for open buy orders 1608253130
status: NEW
Checking for open buy orders 1608253131
status: NEW
Checking for open buy orders 1608253133
status: NEW
Checking for open buy orders 1608253135
status: NEW


status: NEW
Checking for open buy orders 1608253129
status: NEW
Checking for open buy orders 1608253130
status: NEW
Checking for open buy orders 1608253131
status: NEW
Checking for open buy orders 1608253133
status: NEW
Checking for open buy orders 1608253135
status: NEW
Checking for open buy orders 1608253136
status: NEW
Checking for open buy orders 1608253127
status: NEW
Checking for open buy orders 1608253128
status: NEW
Checking for open buy orders 1608253129
status: NEW
Checking for open buy orders 1608253130
status: NEW
Checking for open buy orders 1608253131
status: NEW
Checking for open buy orders 1608253133
status: NEW
Checking for open buy orders 1608253135
status: NEW
Checking for open buy orders 1608253136
status: NEW
Checking for open buy orders 1608253127
status: NEW
Checking for open buy orders 1608253128
status: NEW
Checking for open buy orders 1608253129
status: NEW
Checking for open buy orders 1608253130
status: NEW
Checking for open buy orders 1608253131
status: NEW


status: NEW
Checking for open buy orders 1608253127
status: NEW
Checking for open buy orders 1608253128
status: NEW
Checking for open buy orders 1608253129
status: NEW
Checking for open buy orders 1608253130
status: NEW
Checking for open buy orders 1608253131
status: NEW
Checking for open buy orders 1608253133
status: NEW
Checking for open buy orders 1608253135
status: NEW
Checking for open buy orders 1608253136
status: NEW
Checking for open buy orders 1608253127
status: NEW
Checking for open buy orders 1608253128
status: NEW
Checking for open buy orders 1608253129
status: NEW
Checking for open buy orders 1608253130
status: NEW
Checking for open buy orders 1608253131
status: NEW
Checking for open buy orders 1608253133
status: NEW
Checking for open buy orders 1608253135
status: NEW
Checking for open buy orders 1608253136
status: NEW
Checking for open buy orders 1608253127
status: NEW
Checking for open buy orders 1608253128
status: NEW
Checking for open buy orders 1608253129
status: NEW


...

status: NEW
Checking for open buy orders 1608461512
status: NEW
Checking for open buy orders 1608466192
status: NEW
Checking for open buy orders 1608475191
status: NEW
Checking for open buy orders 1608625249
status: NEW
Checking for open buy orders 1608253135
status: NEW
Checking for open buy orders 1608253136
status: NEW
Checking for open buy orders 1608387630
status: NEW
Checking for open buy orders 1608449923
status: NEW
Checking for open buy orders 1608461512
status: NEW
Checking for open buy orders 1608466192
status: NEW
Checking for open buy orders 1608475191
status: NEW
Checking for open buy orders 1608625249
status: NEW
Checking for open buy orders 1608253135
status: NEW
Checking for open buy orders 1608253136
status: NEW
Checking for open buy orders 1608387630
status: NEW
Checking for open buy orders 1608449923
status: NEW
Checking for open buy orders 1608461512
status: NEW
Checking for open buy orders 1608466192
status: NEW
Checking for open buy orders 1608475191
status: NEW


status: NEW
Checking for open buy orders 1608387630
status: NEW
Checking for open buy orders 1608449923
status: NEW
Checking for open buy orders 1608461512
status: NEW
Checking for open buy orders 1608466192
status: NEW
Checking for open buy orders 1608475191
status: NEW
Checking for open buy orders 1608625249
status: NEW
Checking for open buy orders 1608253135
status: NEW
Checking for open buy orders 1608253136
status: NEW
Checking for open buy orders 1608387630
status: NEW
Checking for open buy orders 1608449923
status: NEW
Checking for open buy orders 1608461512
status: NEW
Checking for open buy orders 1608466192
status: NEW
Checking for open buy orders 1608475191
status: NEW
Checking for open buy orders 1608625249
status: NEW
Checking for open buy orders 1608253135
status: NEW
Checking for open buy orders 1608253136
status: NEW
Checking for open buy orders 1608387630
status: NEW
Checking for open buy orders 1608449923
status: NEW
Checking for open buy orders 1608461512
status: NEW


status: NEW
Checking for open buy orders 1608253135
status: NEW
Checking for open buy orders 1608253136
status: NEW
Checking for open buy orders 1608387630
status: NEW
Checking for open buy orders 1608449923
status: NEW
Checking for open buy orders 1608461512
status: NEW
Checking for open buy orders 1608466192
status: NEW
Checking for open buy orders 1608475191
status: NEW
Checking for open buy orders 1608625249
status: NEW
Checking for open buy orders 1608253135
status: NEW
Checking for open buy orders 1608253136
status: NEW
Checking for open buy orders 1608387630
status: NEW
Checking for open buy orders 1608449923
status: NEW
Checking for open buy orders 1608461512
status: NEW
Checking for open buy orders 1608466192
status: NEW
Checking for open buy orders 1608475191
status: NEW
Checking for open buy orders 1608625249
status: NEW
Checking for open buy orders 1608253135
status: NEW
Checking for open buy orders 1608253136
status: NEW
Checking for open buy orders 1608387630
status: NEW


status: NEW
Checking for open buy orders 1608475191
status: NEW
Checking for open buy orders 1608625249
status: NEW
Checking for open buy orders 1608253135
status: NEW
Checking for open buy orders 1608253136
status: NEW
Checking for open buy orders 1608387630
status: NEW
Checking for open buy orders 1608449923
status: NEW
Checking for open buy orders 1608461512
status: NEW
Checking for open buy orders 1608466192
status: NEW
Checking for open buy orders 1608475191
status: NEW
Checking for open buy orders 1608625249
status: NEW
Checking for open buy orders 1608253135
status: NEW
Checking for open buy orders 1608253136
status: NEW
Checking for open buy orders 1608387630
status: NEW
Checking for open buy orders 1608449923
status: NEW
Checking for open buy orders 1608461512
status: NEW
Checking for open buy orders 1608466192
status: NEW
Checking for open buy orders 1608475191
status: NEW
Checking for open buy orders 1608625249
status: NEW
Checking for open buy orders 1608253135
status: NEW


KeyboardInterrupt: 